## Prettify the page with some `css`

In [1]:
%%js
$(".repository-content, .container").css("width", "90%");
$("body, #outer-dashboard, .navbar-default").css("background-color", "papayawhip");
$(".navbar-default").css("background-color", "lightblue");

<IPython.core.display.Javascript object>

In [2]:
%%html
<img src="//localhost:3010/files/img/pydata.png">

In [3]:
from ipywidgets import widgets, interact, interactive
from IPython.display import display

In [4]:
debug_widget = widgets.HTML('<h3 class="pulsate">Loading IMDB data...</h3>')
display(debug_widget)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


## Display the `Loading` banner

In [5]:
%%js

$(function() {
  function pulsate() {
    $(".pulsate").
      animate({opacity: 0.2}, 500, 'linear').
      animate({opacity: 1}, 500, 'linear', pulsate);
  }
  pulsate();
});

<IPython.core.display.Javascript object>

## Now can do more expensive imports

In [6]:
import numpy as np
import pandas as pd
import json
import pandas.io.sql as psql
import sqlite3 as sql
from ipykernel.comm import Comm

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, LassoSelectTool, CustomJS
from bokeh.io import output_notebook, push_notebook, show
from bokeh.sampledata.movies_data import movie_path

In [7]:
output_notebook(hide_banner=True)

In [8]:
wallet = type('Wallet', (), {})

In [9]:
genres = [
    "All",
    "Action",
    "Adventure",
    "Animation",
    "Biography",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Family",
    "Fantasy",
    "History",
    "Horror",
    "Music",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Short",
    "Sport",
    "Thriller",
    "War",
    "Western",
]

In [10]:
query = """
SELECT omdb.ID,
       imdbID,
       Title,
       Year,
       omdb.Rating as mpaaRating,
       Runtime,
       Genre,
       Released,
       Director,
       Writer,
       omdb.Cast,
       imdbRating,
       imdbVotes,
       Language,
       Country,
       Oscars,
       tomatoes.Rating as numericRating,
       Meter,
       Reviews,
       Fresh,
       Rotten,
       userMeter,
       userRating,
       userReviews,
       BoxOffice,
       Production
FROM omdb, tomatoes
WHERE omdb.ID = tomatoes.ID AND Reviews >= 10
"""

In [11]:
conn = sql.connect(movie_path)
movies = psql.read_sql(query, conn)

In [12]:
movies["color"] = np.where(movies["Oscars"] > 0, "purple", "steelblue")
movies["alpha"] = np.where(movies["Oscars"] > 0, 0.9, 0.25)
movies.fillna(0, inplace=True)  # just replace missing values with zero
movies["revenue"] = movies.BoxOffice.apply(lambda x: '{:,d}'.format(int(x)))

In [13]:
axis_map = {
    "Tomato Meter": "Meter",
    "Numeric Rating": "numericRating",
    "Number of Reviews": "Reviews",
    "Box Office (dollars)": "BoxOffice",
    "Length (minutes)": "Runtime",
    "Year": "Year",
}

In [14]:
source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], revenue=[], alpha=[]))

hover = HoverTool(tooltips=[
    ("Title", "@title"),
    ("Year", "@year"),
    ("$", "@revenue")
])

plot = figure(plot_height=600, plot_width=700, title="", toolbar_location='right', tools='lasso_select,reset')
plot.add_tools(hover)
plot.circle(x="x", y="y", source=source, size=7, color="color", line_color=None, fill_alpha="alpha")
plot.background_fill_alpha = 0
plot.border_fill_alpha = 0

## Some `javascript` to copy selected points elsewhere and send them to backend

In [15]:
source.callback = CustomJS(code=r"""
    // cb_obj is a magic name
    var idx = cb_obj.selected['1d'].indices;
    var ids = [];
    for (i = 0; i < idx.length; i++) {
        ids.push(idx[i]);
    }
    window.comm_handle.send(JSON.stringify(ids));
    console.log("Kernel: enjoy your data!");
""")

In [16]:
handle = show(plot, notebook_handle=True)

In [17]:
debug_widget.value = ''

In [18]:
table_widget = widgets.HTML('<hr>')
display(table_widget)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [19]:
def select_movies(reviews, year, oscars, boxoffice, genre, x_axis, y_axis, director, cast):
    director_val = director.strip()
    cast_val = cast.strip()
    selected = movies[
        (movies.Reviews >= reviews) &
        (movies.BoxOffice >= (boxoffice * 1e6)) &
        (movies.Year >= year[0]) &
        (movies.Year <= year[1]) &
        (movies.Oscars >= oscars)
    ]
    if (genre != "All"):
        selected = selected[selected.Genre.str.contains(genre)==True]
    if (director_val != ""):
        selected = selected[selected.Director.str.contains(director_val)==True]
    if (cast_val != ""):
        selected = selected[selected.Cast.str.contains(cast_val)==True]
    return selected

In [20]:
reviews = widgets.IntSlider(description="Reviews", value=80, min=10, max=300, step=10, continuous_update=False)
year = widgets.IntRangeSlider(description="Year", min=1940, max=2014, value=(1980, 2010), step=1, continuous_update=False)
oscars = widgets.IntSlider(description="Oscars", min=0, max=4, value=0, step=1, continuous_update=False)
boxoffice = widgets.IntSlider(description="BoxOffice, $m", min=0, max=800, value=0, step=1, continuous_update=False)

In [21]:
genre = widgets.Dropdown(description="Genre", value="All", options=genres)
x_axis = widgets.Dropdown(description="X Axis", options=sorted(axis_map.keys()), value="Tomato Meter")
y_axis = widgets.Dropdown(description="Y Axis", options=sorted(axis_map.keys()), value="Number of Reviews")

In [22]:
director = widgets.Text(description="Director", placeholder="Director name contains")
cast = widgets.Text(description="Cast", placeholder="Cast names contains")

In [23]:
@interact(reviews=reviews, year=year, oscars=oscars, boxoffice=boxoffice, 
          genre=genre, x_axis=x_axis, y_axis=y_axis, director=director, cast=cast)
def update(reviews, year, oscars, boxoffice, genre, x_axis, y_axis, director, cast):
    df = select_movies(reviews, year, oscars, boxoffice, genre, x_axis, y_axis, director, cast)
    wallet.index = df.index
    x_name = axis_map[x_axis]
    y_name = axis_map[y_axis]

    plot.xaxis.axis_label = x_axis
    plot.yaxis.axis_label = y_axis
    plot.title.text = "{} movies selected".format(len(df))
    source.data = dict(
        x=df[x_name],
        y=df[y_name],
        color=df["color"],
        title=df["Title"],
        year=df["Year"],
        revenue=df["revenue"],
        alpha=df["alpha"],
    )
    push_notebook(handle=handle)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [24]:
%%js
Jupyter.notebook.kernel.comm_manager.register_target(
    'foobar42', 
    function (comm, msg) {
        console.log("Registered Jupyter comms");
        window.comm_handle = comm;
        comm.on_msg(function (msg) {console.log(msg.content.data)})
    }
)

<IPython.core.display.Javascript object>

In [25]:
comm = Comm(target_name='foobar42')

def payload_to_df(payload=None):
    indices = json.loads(payload or wallet.payload)
    return movies.loc[wallet.index[indices]][:20][['Title', 'Year']]

@comm.on_msg
def handle_msg(msg):
    wallet.payload = msg['content']['data']
    df = payload_to_df(wallet.payload)
    table_widget.value = df.to_html(index=False, classes="table table-striped table-hover table-responsive")

In [26]:
%%js
$(".widget-label").css("min-width", "120px");
$(".widget-hbox").css("width", "400px");

<IPython.core.display.Javascript object>